# 1. Encoder-decoder architecture

We're going to discuss a very important technique in neural networks. We are going to speak about encoder-decoder architecture and about attention mechanism. We will cover them by the example of neural machine translation, just because they were mostly proposed for machine translation originally. 

But now they are applied to many, many other tasks. For example, you can think about summarization or simplification of the texts, or sequence to sequence chatbots and many, many others. Now let us start with the general idea of the architecture. 

![eda_1.png](pics/eda_1.png)

We have some sequence as the input, and we would want to get some sequence as the output. For example, this could be two sequences for different languages, right? We have our encoder and the task of the encoder is to build some hidden representation over the input sentence in some hidden way. So we get this green hidden vector that tries to **encode the whole meaning of the input sentence**. Sometimes this vector is also called **thought vector**, because it encodes the thought of the sentence. The encoder task is to decode this thought vector or context vector into some output representation. For example, the sequence of words from the other language.

Now what types of encoders could we have here? Well, one most obvious type would be her current neural networks, but actually this is not the only option. So be aware that we have also convolutional neural networks that can be very fast and nice, and they can also encode the meaning of the sentence. We could also have some hierarchical structures. For example, recursive neural networks try to use syntax of the language and build the representation hierarchically from from bottom to the top, and understand the sentence that way. 

![eda_1.png](pics/eda_2.png)

Okay, now what is the first example of sequence to sequence architecture? This is the model that was proposed in 2014 and it is rather simple. So it says, we have some LCM module or RNN module that encodes our input sentence, and then we have end of sentence token at some point. At this point, we understand that our state is our thought vector or context vector, and we need to decode starting from this moment. The decoding is conditional language modelling. So you're already familiar with language modelling with neural networks, but now it is conditioned on this context vector, the green vector.

Okay, as any other language model, you usually fit the output of the previous state as the input to the next state, and generate the next words just one by one.

Now, let us go deeper and stack several layers of our LSTM model. You can do this straightforwardly like this.

![eda_3.png](pics/eda_3.png)

So let us move forward, and speak about a little bit different variant of the same architecture. 

![eda_3.png](pics/eda_4.png)

One problem with the previous architectures is that the green context letter can be forgotten. So if you only feed it as the inputs to the first state of the decoder, then you are likely to forget about it when you come to the end of your output sentence. So it would be better to feed it at every moment. And this architecture does exactly that, it says that every stage of the decoder should have three kind of errors that go to it. 

1. the error from the previous state, 
2. the error from this context vector,
3. the current input which is the output of the previous state. 

Okay, now let us go into more details with the formulas. 

![eda_5.png](pics/eda_5.png)

So you have your sequence modeling task conditional because you need to produce the probabilities of one sequence given another sequence, and you factorize it using the chain rule. 

Also importantly you see that $x$ variables are not needed anymore because you have encoded them to the $v$ vector. V vector is obtained as the last hidden state of the encoder, and encoder is just recurrent neural network. 

The decoder is also the recurrent neural network. However, it has more inputs, right? So you see that now I concatenate the current input Y with the V vector. And this means that I will use all kind of information, all those three errors in my transitions.

Now, how do we get predictions out of this model? Well, the easiest way is just to do soft marks, right? So when you have your decoder RNN, you have your hidden states of your RNN and they are called $S_j$.

You can just apply some linear layer, and then softmax, to get the probability of the current word, given everything that we have, awesome. 

![eda_5.png](pics/eda_6.png)


Now let us try to see whether those v vectors are somehow meaningful. One way to do this is to say, okay they are let's say three dimensional hidden vectors. Let us do some dimensional reduction, for example, by TS&E or PCA, and let us plot them just by two dimensions just to see what are the vectors. 

So you see that the representations of some sentences are close here and it's nice that the model can capture that active and passive voice doesn't actually matter for the meaning of the sentence. For example, you see that the sentence, I gave her a card or she was given a card are very close in this space. 
![eda_5.png](pics/eda_7.png)

Okay, even though these representations are so nice, this is still a bottleneck. So you should think about how to avoid that. And to avoid that, we will go into attention mechanisms and this will be the topic of our next chapter. 

# 2. Attention mechanism

Attention mechanism is a super powerful technique in neural networks. So let us cover it first with some pictures and then with some formulas. 

![eda_8.png](pics/eda_8.png)

Just to recap, we have some encoder that has $h$ states and decoder that has some $s$ states. Now, let us imagine that we want to produce the next decoder state. So we want to compute $s_j$. How can we do this? 

In the previous chapter, we just used the $v$ vector, which was the information about the whole encoded input sentence. And instead of that, we could do something better. We can look into all states of the encoder with some weights. So these alphas denote some weights that will say us whether it is important to look there or here. How can we compute this alphas? 

Well, we want them 
1. to be **probabilities**, 
2. we want them to **capture some similarity** between our current moment in the decoder and different moments in the encoder. 

This way, we'll look into more similar places, and they will give us the most important information to go next with our decoding. 

![eda_9.png](pics/eda_9.png)

If we speak about the same thing with the formulas, we will say that, now, instead of just one $v$ vector that we had before, we will have $v_j$, which is different for different positions of the decoder. And this $v_j$ vector will be computed as an average of encoder states. And the weights will be computed as soft marks because they need to be probabilities. And this soft marks will be applied to similarities of encoder and decoder states. 

Now, do you have any ideas how to compute those similarities? 

![eda_9.png](pics/eda_10.png)

I have a few. So papers actually have tried lots and lots of different options, and there are just three options for you to try to memorize. 

Maybe the easiest option is in the bottom. Let us just do dot product of encoder and decoder states. It will give us some understanding of their similarity. Another way is to say, maybe we need some weights there, some metrics that we need to learn, and it can help us to capture the similarity better. This thing is called multiplicative attention. And maybe we just do not want to care at all with our mind how to compute it. We just want to say, "Well, neural network is something intelligent. Please do it for us." And then we just take one layer over neural network and say that it needs to predict these similarities. So you see there that you have $h$ and $s$ multiplied by some matrices and summed. That's why it is called additive attention. And then you have some non-linearity applied to this. 

These are three options, and you can have also many more options. Now, let us put all the things together, just again to understand how does attention works. 

![eda_11.png](pics/eda_11.png)

You have your conditional language modeling task. You'll try to predict $Y$ sequence given $s$ sequence. And now, you encode your $x$ sequence to some $v_j$ vector, which is different for every position. This $v_j$ vector is used in the decoder. It is concatenated with the current input of the decoder. And this way, the decoder is aware of all the information that it needs, the previous state, the current input, and now, this specific context vector, computed especially for this current state. 

Now, let us see where the attention works. 
![eda_12.png](pics/eda_12.png)

So neural machine translation had lots of problems with long sentences. You can see that blue score for long sentences is really lower, though it is really okay for short ones. Neural machine translation with attention can solve this problem, and it performs really nice for even long sentences. 

Well, this is really intuitive because **attention helps to focus on different parts of the sentence when you do your predictions**. And for long sentences, it is really important because, otherwise, you have to encode the whole sentence into just one vector, and this is obviously not enough. Now, to better understand those $alpha_{ij}$ ways that we have learned with the attention, let us try to visualize them. 

![eda_13.png](pics/eda_13.png)

This weights can be visualized with $i$ by $j$ matrices. Let's say, what is the best promising place in the encoder for every place in the decoder? So with the light dot here, you can see those words that are aligned. So you see this is a very close analogy to word alignments that we have covered before. 

We just learn that these words are somehow similar, relevant, and we should look into this once to translate them to another language. And this is also a good place to note that we can use some techniques from traditional methods, from words alignments and incorporate them to neural machine translation. For example, priors for words alignments can really help here for neural machine translation. 

Now, do you think that this attention technique is really similar to how humans translate real sentences? I mean, humans also look into some places and then translate this places. They have some attention. Do you see any differences? 

![eda_14.png](pics/eda_14.png)

Well, actually there is one important difference here. 

So humans save time with attention because they look only to those places that are relevant. On the contrary, here, we waste time because to guess what is the most relevant place, we first need to check out all the places and compute similarities for the whole encoder states. And then just say, "Okay, this piece of the encoder is the most meaningful." 

![eda_15.png](pics/eda_15.png)

Now, the last story for this chapter is how to make this attention save time, not waste time. It is called local attention, and the idea is rather simple. We say, let us first time try to predict what is the best place to look at. And then after that, **we will look only into some window around this place**. And we will not compute similarities for the whole sequence. 

Now, first, how you can predict the best place. One easy way would be to say, "You know what? Those matrices should be strictly diagonal, and the place for position $j$ should be $j$." Well, for some languages, it might be really bad if you have some different orders and then you can try to predict it. How do you do this? You have this sigmoid for something complicated. This sigmoid gives you probability between zero to one. And then you scale this by the length of the input sentence $I$. So you see that this will be indeed something in between zero and I, which means that you will get some position in the input sentence. Now, what is inside those sigmoid? Well, you see a current decoder state $s_j$, and you just apply some transformations as usual in neural networks. Anyway, so when you have this $a_j$ position, you can just see that you need to look only into this window and compute similarities for attention alphas as usual, or you can also try to use some Gaussian to say that actually those words that are in the middle of the window are even more important. So you can just multiply some Gaussian priors by those alpha weights that we were computing before. 

![eda_16.png](pics/eda_16.png)

Now, I want to show you the comparison of different methods. You can see here that we have **global attention** and **local attention**. And for local attention, we have monotonic predictions and predictive approach. And the last one performs the best. Do you remember what is inside the brackets here? These are different ways to compute similarities for attention weights. So you remember dot product and multiplicative attention? And, also, you could have location-based attention, which is even more simple. It says that we should just take $s_j$ and use it to compute those weights. This is all for that presentation, and I am looking forward to see you in the next one.

# 3. Hot to deal with a vocabulary? 

This chapter is about all kind of problems that you can have with vocabulary in machine translations.
![eda_17.png](pics/eda_17.png)

So first, vocabulary is usually too large, and it is too long to compute softmax. 

Second, vocabulary is usually too small, and you have **out-of-vocabulary words**, and you need somehow to deal with them. And you have a whole range on different tricks for you in neural language modeling and in machine translation.

Now, let us start with hierarchical softmax. It is a nice procedure to help you to compute softmax in a fast way.

![eda_17.png](pics/eda_18.png)

So the idea is to build the binary tree for your words, like here. And the words in this tree will have some codes. 
![eda_17.png](pics/eda_19.png)

So for example, the zebra will have code 01, which means that first, we go to the left, it is 0, and then we go to the right, it is 1. And importantly, it is just unique mapping between words and codes in the tree.

So we are going to use this property, and we are going to produce the probabilities for words using their binary codes. 

![eda_17.png](pics/eda_20.png)

This is how we make it. So instead of computing the probability of the word and normalizing it across the whole vocabulary, we are going to split it into separate terms. Each term is the probability of the digit, is the probability of the next decision in the tree, whether to go to the left or to the right. And we build these probabilities into the product so that this product is going to estimate the probability of the whole word.

Now, do you believe that this product of binary probabilities is going to be normalized into one across the whole vocabulary?

Well, sounds like a magic, isn't it? But let's see that actually, it happens. 

![eda_21.png](pics/eda_21.png)

So you see that this is your binary tree, and you see those probabilities. They are just written down for every path in the tree.

Now, you can try to sum the first two probabilities, and you note that 0.1 plus 0.9 gives 1, and this is not just random. This is always 1, just because this is probability for two options, right, going to the left and going to the right. That's why they are necessary summed into 1. So if you sum those things, you get their common prefix, which will be 0.7 by 0.8. Then you can try to sum two other childs in the tree, and you get, again, their common prefix. You sum two other childs, and you get, again, their common prefix. And finally, you sum these two values, and you get 1. So it is clearly seen that if you go from bottom to the top with this tree, you will get probability equal to 1. 

![eda_22.png](pics/eda_22.png)

So just to sum it up, we split the probability into these binary solutions, and we use some tree to do this. What kind of tree do we do? Well, it is actually not so important. So interestingly, even just random tree can help. Alternatively, you can use Huffman tree that gets to know the frequencies of some words, and it uses that. Or you can try to use some semantics so you can just cluster your words in the data based on their similarities. Or you can use some pre-built structure that says that these words are similar and that they should be found in one hierarchy branch.

Okay, now I have one last question for you for this method. What would be a way to model the probability of $d_i$ given all the other stuff?

So usually, we have softmax to model some probabilities. What would be the probability in this case?

Well, in this case, we have just only two options, left and right. So instead of softmax, we can have sigmoid function, which has just two outcomes.

Okay, now let us come back to some other problems with our bot vocabulary. 

![eda_23.png](pics/eda_23.png)

And copy mechanism is something to help with out-of-vocabulary words. Imagine you have some sentence, and some words are UNK tokens. So you do not have them in the vocabulary, and you do not know how to translate them, you get UNK tokens as the result.

But what if you know the word alignments? If you know how the words are aligned, you can use that. So you can say, okay, this UNK corresponds to that UNK, which corresponds to this source word. Let us just do dictionary translation, or let us just copy, because this is the name of some place or some other name. Let us just copy this as is. This is why it is called **copy mechanism**, and the algorithm is super simple. 

![eda_24.png](pics/eda_24.png)

So you need first to make sure that you somehow learn word alignments. For example, your neural machine translation system has these alignments as an input. And it tries to predict them along with the translation predictions.

Now, you get your translation with UNK tokens, and you post-process this. So you just copy the source words, or you translate them with dictionary or do whatever else what you want. Okay, very simple and nice technique, but actually, there are still many problems. 

![eda_25.png](pics/eda_25.png)

For example, you can have some multi-word alignments. What if the morphology of the languages are complicated, and probably, you want to split it somehow into some parts? Or what if you have some informal spelling? All those things are usually out-of-vocabulary words. And these examples show you that sometimes, it is very nice to go to sub-word level. 

For example, for rich morphology, it would be nice to model every piece of the word independently.

Or for informal spelling, it would be definitely good to model them by letters. Because there is no chance to find these words as a whole in the vocabulary. 

![eda_25.png](pics/eda_26.png)

Okay, so there are two big trends in sub-word modelling. One big trend is to do some hybrid models that somehow combines word-level models and character-level models. Another big trend is to do the same architecture, let's say, recurrent neural network, but with some small units, something in between words and characters. So this is one architecture, but other units.

Okay, let us start with hybrid models.

![eda_27.png](pics/eda_27.png)

So you might know that sometimes, character-based models are super useful. For example, you see the word drinkable. And if you can build your convolutional neural network that has some convolutions that represent the meaning of drink. And then some other convolutions that represent the meaning of able, you are likely to build the meaning of the whole word, even if you have never seen that. So character-level convolutional neural networks can work fine. 

Also, bidirectional LSTMs can be used on word level as well as on character level.

Now, for our translation model, it is super nice to have hybrid models. Let's say, let us first try to work on word level.

![eda_28.png](pics/eda_28.png)

So let us try to produce word translations.

For example, we have a cute cat here. And a and cat are nice words. But what if cute is out of vocabulary? We cannot model it on word level. In this case, let us have some separate units, some separate architecture, maybe even completely different, that will model these probabilities for this word on character level, and the same for the decoder. So first, we will try to decode the sequence on word level. And then, in some moments, the decoder will say, okay, this is some UNK token, please do something about it. And then the character-level model will switch on and do something about it.

So this is a very nice and obvious idea, and it is used a lot.

Now, the second big trend would be sub-word modeling, and one good example of that is byte-pair encoding.

![eda_28.png](pics/eda_29.png)

Let us understand what it is. So imagine you have some sentence, and you want to encode this, and you are not quite sure yet what is your vocabulary. You have just some constraints for the size of the vocabulary. So you start with characters. Everything is split to single characters. Then you say, okay, what are the most popular bigrams of my letters here?

Well, I see that S-H happens three times. So maybe I should collapse them into just one unit. And you do this, and you have some other vocabulary right now. Okay, now you say, okay, what is next? Next, I see that these two letters occur a lot, let us collapse them. These two letters also should be collapsed. And then importantly, you can apply the same procedure to sub-word units. So here, you would collapse your bigrams and unigrams into trigrams. And actually, you can stop whenever you want.

So you can proceed until you get the nice size of the vocabulary that you like.

Yep, I have just said you this. So one thing to mention is how to apply this method for test data. So if you have test data, you also split it to letters first. And then you know the exact rules from your training procedure. And you apply those rules to test data, to collapse all the sub-word units as needed.

![eda_28.png](pics/eda_30.png)

Awesome, so this chart shows you why it is so cool technique. So this is the vocabulary size, and this line there, this vertical line, is about the size of the vocabulary that we are allowed. And in case of words, usually you have some long tail that goes outside of this allowed amount of words. But with byte-pair encoding, you can do exactly this number of units, because you decide which rules to apply.

![eda_28.png](pics/eda_31.png)

And finally, I can show you that this actually works fine. So for some different pairs of languages, for some different tasks from, for example, WMT, which is Workshop on Machine Translation, you can see that byte-pair encoding has better BLEU score than word-based techniques. And actually, this BLEU score improvement is very meaningful. This one or two points of BLEU is a lot. So please use this very nice technique if you someday need to build machine translation system.

# 3. How to implement a conversational chat-bot?

 In this chapter we are going to discuss how to apply encode or decode attention architecture to a hot topic nowadays, to chat-bots. 
 
 ![eda_32.png](pics/eda_32.png)
 
First of all, let us understand what do we mean by chat-bots? As for any hot topic, every one means something slightly different. So first we can have goal-oriented bots, and just bots that have a nice conversation with us. 

So for goal-oriented bots, we usually speak about some narrow domain and specific tasks to fulfill. For example, this can be a bot in a call center in our bank that can help customers with their needs. Worth is what can serve some specific answers to some specific questions of the customers. It means that usually these bots are based on some retrieval approach, for example, we could have some database of information, and then it'll just solve those answers for the users. Now, entertaining bots or bots that can just have some conversation with us. I usually call chit chat bots, and the model for them would be **generative**. 

What do I mean by generative models.? 

![eda_32.png](pics/eda_33.png)

It means that we can just generate new responses. Just to compare for retrieval-based models, we would get some predefined responses just ranked from some repository. Or we would have some patterns, and then we would just use these patterns to get some specific replies. 

Now, there are some pros and cons of two models, obviously for generated models you have more freedom, you can generate whatever you want. But you can also have some mistake, and it is more complicated to build these models. 

So in this chapter, we'll speak only about what generative models for conversational chat-bots. And those bots that have some goals to assist users with their specific needs, will be more covered in the next week.

![eda_34.png](pics/eda_34.png)

Okay, so let us just recap that we could have our encoder to encode the incoming message for the bot, and decoder that would generate the bot's response. And we can have, for example, LSTMs in encoder and decoder, and we would also want to have attention.


One other alternative for attention would be to have reversed input. Actually, it is a very simple thing that had been studies before attention. So, we shall say, maybe we need to reverse the input of the sentence, and then our thought vector of the sentence will be somehow more meaningful. Why? To understand that, let us cover one technical detail that we need to understand to built any system of that kind.

![eda_35.png](pics/eda_35.png)

The detail is about a set effect that all the sequences have different lengths. Okay, so you have some questions, for example, and you need to somehow pad those questions to have some fixed length for all of them. Why? Just because you need to pack them into batches and send these batches to your neural network for your stochastic gradient descent. So it suggests some technical implementation detail that if you implement as static recurrent neural network, you need to pad your sequences with some pad tokens.

So if you do this in the end of the sequence, you could see that the end the sequence is absolutely not meaningful, right? It is just PAD, PAD, PAD, and so on. So if you try to build your thought vector base on it, maybe it will not be nice. That's where you just reverse everything, and then you have your words in the end of the sequence.

Now you can encode to that, and the decoder will get some answer for you also pad it as news.

![eda_35.png](pics/eda_36.png)

One other idea would be to do bucketing. What it means is that let us group our sentences into buckets, based on their length. For example, those sentences that have the length less than five, would go into the first bucket. And then for them, we can just pad them to the length of five.


So this approach will give us an opportunity to have not that many pad tokens. Because we will have the adaptable length, based on the maximum length in each bucket.

The only important thing is to put these buckets into different batches then, just to make sure that the recurrent neural network will not get buckets with a different lengths in one of the same batch.

Okay, in the rest of the chapter, I'm going to show you how the chat-bots work, but we will also discuss how they do not work. So you'll see some analysis over the problems, and just a very ideas of what can be fixed.

![eda_38.png](pics/eda_38.png)

So, you can see that this is a human tool machine talk, trained for movie subtitles, and it is rather impressive. So, what is the purpose of living? And the machine says, to live forever, okay sounds good. 

But you can also notice that it is very dependent on the data set that we use to train the model. So if we try to use the same model, lets see for assistant in a bank, maybe that's not a good idea. So there responses will be too dramatic and the topics might be unrealistic. So it is very important to understand that you have some specific properties of the outcome, based on the domain of the data. 

![eda_39.png](pics/eda_39.png)

Now, if you want to use it for calls, let us train it on calls. So you have some meaningful lexis here, but it is inconsistent. So the chat-bot says, what is the operating system of your machine? And the user says linux. After that, just in a few turns, the machine says again, so is it a windows machine? And the user has to answer again. So this is not nice, the bot doesn't remember what was happening before in our conversation, and you can try to fix that. 

So there is a paper that says, you would need to track the intent and the context of the conversation with some separate recurrent neural network. And then we can somehow just memorize for the bot that these topics have been already covered. And we do not need to ask again what is the operating system of the machine, let's say. So you do not see such problems for this example of the dialogue.

![eda_40.png](pics/eda_40.png)

Now another important problem is that the bot has no personality. So if you try to ask the bot, where do you live now? The bot can say, I live in Los Angeles, and that sounds okay. But then if you ask the bot again you will get some other responses. Just because it was trained on the data of questions and answers, and the bot doesn't know about consistency in them. So, one idea would be to build **persona-based models**. It means that, we need to memorize that the bot has some personality. So we just train iton some coherent pieces of dialogues from different persons, and we built this knowledge of persons. So that when we ask what is the country, what is the city, we still get coherent responses.

![eda_41.png](pics/eda_41.png)

Now, another problem is diversity in the responses. So this is a smart reply technology by Google, that says that it can help you to answer Gmail automatically. For example, if you see the email, let us meet up and discuss, you want to get some proposed responses. And the model would propose how about tomorrow, what about tomorrow, I suggest tomorrow, so there is no diversity. And the user cannot pick one of them because all of them are about the same thing.

Also another problem would be that you have two popular responses that can come to any email. Again, you will all have not enough diversity and you will have I love you, even before some email from a colleague that is not good for your chat-bot. 

So how can we cope with that? Do you have any idea how may be to track that? 

![eda_42.png](pics/eda_42.png)

One idea would be to do intent clustering for our responses. For example, we can have some small supervised data, about the types of the responses. For example, you can have the label, how about time for how to how about Friday or something like that. So you have actually some graph of different responses, and you have similarities between the responses built by some bearings, or some distributional semantics model. 

Now, you have some labeled nodes in this graph from your supervised data, and you want to propagate this knowledge to other labels of the graph. So this technique is called **label propagation on graphs**, and **Expander** is a library that implements this technique. So the main idea here is that we will try to propagate the labels of the responses, in such a way that close responses will get close labels. And those in such a way that those labels that already are known from our supervision, will be stayed the same, awesome. 

So the methods can be different but the idea is just to do something clustering, and then to pick up one example from every cluster, and suggest it for the user. 

![eda_43.png](pics/eda_43.png)

So what we get out of it is very nice, so this is the query and this is the top generated responses. And you see that now we have how about Tuesday? I can on Wednesday, I can on some other day, so you see that you have some diversity, and that's what we want you to have.

![eda_43.png](pics/eda_44.png)

Well, even though the bots can try to have some meaningful conversation with you, you can see that there are still so many problems with them. And it is so easy to understand that you are speaking to a bot and not to a human. And that's why actually we should be very careful about that hype. And we should realize that well, indeed, we are very promising and we have some good opportunities in the future, but current models are still not humans.

# 4. Quiz - Encoder-decoder architectures

![eda_quiz](pics/eda_quiz_1.png)
![eda_quiz](pics/eda_quiz_2.png)
![eda_quiz](pics/eda_quiz_3.png)
![eda_quiz](pics/eda_quiz_4.png)
![eda_quiz](pics/eda_quiz_5.png)
![eda_quiz](pics/eda_quiz_6.png)
![eda_quiz](pics/eda_quiz_7.png)
![eda_quiz](pics/eda_quiz_8.png)
